# Modeling & predictions

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_predict, cross_val_score, KFold
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#from sklearn.utils import resample
import xgboost as xgb

### Load data

In [2]:
prep_train = pd.read_csv('data/preprocessed_training_data.csv')

prep_X_train = pd.read_csv('data/preprocessed_X_train.csv')
prep_X_train_scaled = pd.read_csv('data/preprocessed_X_train_scaled.csv')

prep_y_train = pd.read_csv('data/preprocessed_y_train.csv')

orig_X_test = pd.read_csv('data/test.csv')
prep_X_test = pd.read_csv('data/preprocessed_X_test.csv')
prep_X_test_scaled = pd.read_csv('data/preprocessed_X_test_scaled.csv')

orig_X_test2 = pd.read_csv('data/test_2.csv')
test_modified_3 = pd.read_csv('data/preprocessed_X_test2.csv')
prep_X_test2_scaled = pd.read_csv('data/preprocessed_X_test2_scaled.csv')

### Ways to enhance prediction accuracy
- Feature scaling (StandardScaler)
- Tuning of hyperparameters
- Cross-validation
- Removal of outliers

### Helper functions

In [3]:
def split_and_scale_training_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test


def save_predictions_to_csv(predictions):
    results = pd.DataFrame()
    results['Id'] = orig_X_test['Id']
    results['target'] = predictions
    results.to_csv('data/results.csv', index=False)
    return results


def define_X_y_split_and_scale_training_data(data, test_size=0.2, random_state=42):
    X = data.drop('pSat_Pa', axis=1)
    y = data.copy()[['pSat_Pa']]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test


def split_train_data_and_scale_after_feature_engineering(train_data, test_data):
    X_train = train_data.drop('pSat_Pa', axis=1)
    y_train = train_data.copy()[['pSat_Pa']]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(test_data)
    
    return X_train_scaled, X_test_scaled, y_train

# Evaluate R2 values on training data

## Compare basic versions of different models

In [4]:
def test_basic_versions_of_models(X, y, test_size=0.2, random_state=42, use_scaler=True):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    if use_scaler:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    regressors = {
        'Dummy Model': DummyRegressor(),
        'OLS Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(random_state=random_state),
        'SVR': SVR(),
        'Lasso': Lasso(random_state=random_state),
        'XGBRegressor': xgb.XGBRegressor(),
    }

    for name, regressor in regressors.items():
        model = regressor.fit(X_train, y_train.values.ravel())
        predictions = model.predict(X_test)
        r2 = r2_score(y_test, predictions)
        print(f'{name}: R^2 Score = {r2:.4f}')

### Without standard scaler

In [6]:
test_basic_versions_of_models(X=prep_X_train, y=prep_y_train, test_size=0.2, random_state=42, use_scaler=False)

Dummy Model: R^2 Score = -0.0000
OLS Linear Regression: R^2 Score = 0.6957


Random Forest: R^2 Score = 0.7105
SVR: R^2 Score = 0.4713
Lasso: R^2 Score = 0.3295


### With standard scaler

In [7]:
test_basic_versions_of_models(X=prep_X_train, y=prep_y_train, test_size=0.2, random_state=42, use_scaler=True)

Dummy Model: R^2 Score = -0.0000
OLS Linear Regression: R^2 Score = 0.6944


Random Forest: R^2 Score = 0.7106
SVR: R^2 Score = 0.7383
Lasso: R^2 Score = 0.2664


## SVR

### SVR basic

In [5]:
def svr_basic(X_train, y_train, X_test):
    svr = SVR()
    svr.fit(X_train, y_train.values.ravel())
    y_pred = svr.predict(X_test)
    
    return y_pred

#### Test R2

In [9]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7383008493915951


#### Predict on actual test data

In [10]:
y_pred = svr_basic(prep_X_train_scaled, prep_y_train, prep_X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

### Search optimal parameters for SVR using GridSearch (takes a long time!)

In [10]:
def svr_hyperparameter_grid_search(X_train, y_train, X_test):
    param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.2, 0.5]
    }

    svr = SVR()
    grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='r2')
    grid_search.fit(X_train, y_train.values.ravel())

    best_params = grid_search.best_params_

    best_svr = SVR(**best_params)
    best_svr.fit(X_train, y_train.values.ravel())

    y_pred = best_svr.predict(X_test)
    
    return best_params, y_pred

In [11]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

best_params, y_pred = svr_hyperparameter_grid_search(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)

print("Best parameters:", best_params)
print("R2 Score:", r2)

### SVR with hyperparameter tuning

In [6]:
def svr_with_hyperparameter_tuning(X_train, y_train, X_test):
    svr = SVR(C=10, kernel='rbf', gamma='scale', epsilon=0.5) # default C=1, kernel="rbf", gamma="scale", epsilon=0.1
    svr.fit(X_train, y_train.values.ravel())
    y_pred = svr.predict(X_test)
    
    return y_pred

#### Test R2

In [7]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

y_pred = svr_with_hyperparameter_tuning(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7444331502588883


#### Predict on actual test data

In [ ]:
y_pred = svr_with_hyperparameter_tuning(prep_X_train_scaled, prep_y_train, prep_X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

## XGBRegressor

### XGBRegressor basic version

In [10]:
def xgbr_basic(X_train, y_train, X_test):
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    
    return y_pred

#### Test R2

In [11]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

y_pred = xgbr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7299198444669608


#### R2 using cross validation

In [40]:
model = xgb.XGBRegressor()
scores = cross_val_score(model, X_train_scaled, y_train, scoring='r2', cv=5)
print("R2 Score CV:", scores.mean())

R2 Score CV: 0.7243234475265384


#### Predict on actual test data

In [ ]:
y_pred = xgbr_basic(prep_X_train_scaled, prep_y_train, prep_X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

### Search optimal parameters for XGBRegressor using RandomizedSearch

In [12]:
def xgbr_hyperparameter_randomized_search(X_train, y_train, X_test):
    param_dist = {
        'learning_rate': uniform(0.01, 0.3),
        'n_estimators': randint(100, 700),
        'max_depth': randint(3, 10),
        'min_child_weight': randint(1, 20),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.7, 0.3),
        'reg_alpha': uniform(0, 0.5),
        'reg_lambda': uniform(0, 0.5),
        'gamma': uniform(0, 0.5)
    }

    xgbr = xgb.XGBRegressor()

    print("Default Parameters:", xgbr.get_params())
    
    random_search = RandomizedSearchCV(
        xgbr, param_distributions=param_dist, n_iter=20, cv=5, scoring='r2', random_state=42, n_jobs=-1
    )

    random_search.fit(X_train, y_train)
    best_params = random_search.best_params_
    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    print(f"\nmodel = xgb.XGBRegressor(")
    for key, value in best_params.items():
        print(f"    {key}={value},")
    print(")")
    
    return best_params, y_pred

In [16]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

best_params, y_pred = xgbr_hyperparameter_randomized_search(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("\nR2 Score:", r2)

Default Parameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}



R2 Score: 0.7387929452076731

model = xgb.XGBRegressor(
    colsample_bytree=0.9828729111737557,
    gamma=0.16160146601037761,
    learning_rate=0.16563718652300982,
    max_depth=3,
    min_child_weight=17,
    n_estimators=596,
    reg_alpha=0.12695770696717235,
    reg_lambda=0.1234380314193006,
    subsample=0.8481521364198942,
)


### XGBRegressor with hyperparameter tuning

In [14]:
def xgbr_with_hyperparameter_tuning(X_train, y_train, X_test):
    model = xgb.XGBRegressor(
        colsample_bytree=0.9828729111737557,
        gamma=0.16160146601037761,
        learning_rate=0.16563718652300982,
        max_depth=3,
        min_child_weight=17,
        n_estimators=596,
        reg_alpha=0.12695770696717235,
        reg_lambda=0.1234380314193006,
        subsample=0.8481521364198942,
    )
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    
    return y_pred

#### Test R2

In [15]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

y_pred = xgbr_with_hyperparameter_tuning(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7387929452076731


#### Predict on actual test data

In [ ]:
y_pred = xgbr_with_hyperparameter_tuning(prep_X_train_scaled, prep_y_train, prep_X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

## Random forest

### Random forest basic version

In [17]:
def random_forest_basic(X_train, y_train, X_test):
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    
    return y_pred

#### Test R2

In [18]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

y_pred = random_forest_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7105715664218221


#### Predict on actual test data

In [ ]:
y_pred = random_forest_basic(prep_X_train_scaled, prep_y_train, prep_X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

### Search optimal parameters for Random forest using RandomizedSearch

In [19]:
def random_forest_hyperparameter_randomized_search(X_train, y_train, X_test):
    param_dist = {
        'n_estimators': randint(100, 700),
        'max_depth': [None] + list(randint(5, 50).rvs(size=5)),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'max_features': ['auto', 'sqrt', 'log2', None, 0.5],
        'bootstrap': [True, False]
    }

    rf_model = RandomForestRegressor()
    random_search = RandomizedSearchCV(
        rf_model, param_distributions=param_dist, n_iter=10, cv=3, scoring='r2', random_state=42
    )
    random_search.fit(X_train, y_train.values.ravel())

    results = random_search.cv_results_

    for i, (params, r2_score) in enumerate(zip(results['params'], results['mean_test_score'])):
        print(f"Iteration {i + 1}: Parameters = {params}, R2 Score = {r2_score:.4f}")

    best_params = random_search.best_params_
    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    return best_params, y_pred

In [20]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

best_params, y_pred = random_forest_hyperparameter_randomized_search(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)

print("Best parameters:", best_params)
print("R2 Score:", r2)

/home/mmsaisa/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/home/mmsaisa/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/mmsaisa/.local/lib/python3.10/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/home/mmsaisa/.local/lib/python3.10/site-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/home/mmsaisa/.lo

Iteration 1: Parameters = {'bootstrap': True, 'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 15, 'min_samples_split': 12, 'n_estimators': 171}, R2 Score = 0.6548
Iteration 2: Parameters = {'bootstrap': True, 'max_depth': 41, 'max_features': 'sqrt', 'min_samples_leaf': 19, 'min_samples_split': 12, 'n_estimators': 558}, R2 Score = 0.6954
Iteration 3: Parameters = {'bootstrap': False, 'max_depth': 41, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 4, 'n_estimators': 408}, R2 Score = 0.6506
Iteration 4: Parameters = {'bootstrap': False, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 413}, R2 Score = 0.5818
Iteration 5: Parameters = {'bootstrap': False, 'max_depth': 41, 'max_features': None, 'min_samples_leaf': 17, 'min_samples_split': 11, 'n_estimators': 575}, R2 Score = 0.6742
Iteration 6: Parameters = {'bootstrap': False, 'max_depth': 29, 'max_features': 'log2', 'min_samples_leaf': 12, 'min_samples_spli

### Random forest with hyperparameter tuning

In [21]:
def random_forest_with_hyperparameter_tuning(X_train, y_train, X_test):
    model = RandomForestRegressor(random_state=42, n_estimators=500)#, max_depth=20)
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    
    return y_pred

#### Test R2

In [22]:
X_train_scaled, X_test_scaled, y_train, y_test = split_and_scale_training_data(prep_X_train, prep_y_train)

y_pred = random_forest_with_hyperparameter_tuning(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7131669123772304


#### Predict on actual test data

In [ ]:
y_pred = random_forest_with_hyperparameter_tuning(prep_X_train_scaled, prep_y_train, prep_X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

## Further feature engineering

### Filter training data MW > 100

In [23]:
prep_train.shape

(27147, 33)

In [24]:
def filter_mw_gt_100(train):
    return train[train['MW'] > 100]

train_modified_1 = filter_mw_gt_100(prep_train)

train_modified_1.shape

(27125, 33)

#### Test R2

In [25]:
X_train_scaled, X_test_scaled, y_train, y_test = define_X_y_split_and_scale_training_data(train_modified_1)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7516222396593214


In [26]:
y_pred = xgbr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7405315418031904


#### Predict on actual test data

In [ ]:
X_train_scaled, X_test_scaled, y_train = split_train_data_and_scale_after_feature_engineering(train_modified_1, prep_X_test)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

### Drop non-significant (?) features

In [27]:
def drop_non_significant_features(train, test):
    columns_to_drop = [
        'C.C.C.O.in.non.aromatic.ring',
        'aromatic.hydroxyl',
        'category_None',
        'category_apin_decane',
        'category_apin_decane_toluene',
        'category_apin_toluene',
        'category_decane_toluene',
        'nitroester'
    ]

    train_modified = train.drop(columns_to_drop, axis=1)
    test_modified = test.drop(columns_to_drop, axis=1)
    
    return train_modified, test_modified

#### Test R2

In [28]:
train_modified_2, test_modified_2 = drop_non_significant_features(train_modified_1, prep_X_test)

X_train_scaled, X_test_scaled, y_train, y_test = define_X_y_split_and_scale_training_data(train_modified_2)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7529589303538307


In [29]:
y_pred = xgbr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7427340887024483


#### Predict on actual test data

In [ ]:
X_train_scaled, X_test_scaled, y_train = split_train_data_and_scale_after_feature_engineering(train_modified_2, test_modified_2)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

results = save_predictions_to_csv(y_pred)

results.head()

### Create new feature for functional group count

In [33]:
def create_feature_for_func_group_count(train, test):
    functional_groups = ['C.C..non.aromatic.', 'C.C.C.O.in.non.aromatic.ring', 'aldehyde', 'aromatic.hydroxyl',
        'carbonylperoxyacid', 'carbonylperoxynitrate', 'carboxylic.acid',
        'ester', 'ether..alicyclic.', 'hydroperoxide', 'hydroxyl..alkyl.',
        'ketone', 'nitrate', 'nitro', 'nitroester', 'peroxide']

    train_modified = train.copy()
    train_modified['NumOfGroups'] = train_modified[functional_groups].gt(0).sum(axis=1)

    test_modified = test.copy()
    test_modified['NumOfGroups'] = test_modified[functional_groups].gt(0).sum(axis=1)

    return train_modified, test_modified

#### Test R2

In [34]:
train_modified_3, test_modified_3 = create_feature_for_func_group_count(train_modified_2, prep_X_test)
train_modified_3, test_modified_3 = drop_non_significant_features(train_modified_3, test_modified_3)

X_train_scaled, X_test_scaled, y_train, y_test = define_X_y_split_and_scale_training_data(train_modified_3)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.753711166383837


In [35]:
y_pred = xgbr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7452083831447354


### Reduce nitro & nitrate group weight from MW

In [36]:
def reduce_nitro_and_nitrate_weight(train, test):
    train_modified = train.copy()
    train_modified['MW'] = train_modified['MW'] - train_modified['nitro']*46 - train_modified['nitrate']*62

    test_modified = test.copy()
    test_modified['MW'] = test_modified['MW'] - test_modified['nitro']*46 - test_modified['nitrate']*62

    return train_modified, test_modified

#### Test R2

In [38]:
train_modified_4, test_modified_4 = reduce_nitro_and_nitrate_weight(train_modified_3, test_modified_3)

X_train_scaled, X_test_scaled, y_train, y_test = define_X_y_split_and_scale_training_data(train_modified_4)

y_pred = svr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7538312210452933


In [39]:
y_pred = xgbr_basic(X_train_scaled, y_train, X_test_scaled)

r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: 0.7436279157768789
